In [23]:
%pip install SQLAlchemy
%pip install pyppeteer

Note: you may need to restart the kernel to use updated packages.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for websockets: filename=websockets-10.4-cp312-cp312-macosx_10_9_universal2.whl size=100129 sha256=128e099a153b67c1db897c8088d58e81b851500a16aea05edf1cd2c668f30003
  Stored in directory: /Users/ajay/Library/Caches/pip/wheels/80/cf/6d/5d7e4c920cb41925a178b2d2621889c520d648bab487b1d7fd
Successfully built websockets
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine  

In [15]:
engine = create_engine("sqlite:///manga_data.db")  

with engine.connect() as conn, conn.begin():  
  data = pd.read_sql_table("manga", conn)  

In [20]:
data['image_url'][0]

'["https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6663e1c416f25/ch-005/naruto_v01bch005_147.jpg", "https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6663e1c416f25/ch-005/naruto_v01_ch005_128.jpg", "https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6663e1c416f25/ch-005/naruto_v01_ch005_129.jpg", "https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6663e1c416f25/ch-005/naruto_v01_ch005_130.jpg", "https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6663e1c416f25/ch-005/naruto_v01_ch005_131.jpg", "https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6663e1c416f25/ch-005/naruto_v01_ch005_132.jpg", "https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6663e1c416f25/ch-005/naruto_v01_ch005_133.jpg", "https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6663e1c416f25/ch-005/naruto_v01_ch005_134.jpg", "https://www.mangasub.net/wp-content/uploads/WP-manga/data/manga_6663e1c416f25/ch-005/naruto_v

In [ ]:
import os
import sqlite3
import requests
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor

def download_image(image_url, chapter_folder):
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()
        file_name = os.path.basename(urlparse(image_url).path)
        image_path = os.path.join(chapter_folder, file_name)

        with open(image_path, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)

        print(f"Downloaded: {file_name}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {image_url}: {e}")

def process_chapter(manga_name, chapter, image_urls):
    stored_manga_folder = os.path.join("../../stored_manga")
    os.makedirs(stored_manga_folder, exist_ok=True)

    manga_folder = os.path.join(stored_manga_folder, manga_name)
    os.makedirs(manga_folder, exist_ok=True)

    chapter_folder = os.path.join(manga_folder, f"chapter_{chapter:03d}")
    os.makedirs(chapter_folder, exist_ok=True)
    print(f"Downloading images for {manga_name} Chapter {chapter}...")

    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(lambda url: download_image(url, chapter_folder), image_urls)

def fetch_and_download():
    conn = sqlite3.connect('manga_data.db')
    cursor = conn.cursor()

    cursor.execute("SELECT manga_name, chapter, image_url FROM manga")
    rows = cursor.fetchall()

    for row in rows:
        manga_name = row[0].replace(" ", "_")
        chapter = int(row[1])
        image_urls = eval(row[2])
        process_chapter(manga_name, chapter, image_urls)

    conn.close()
    print("All images downloaded successfully!")

if __name__ == "__main__":
    fetch_and_download()


Downloaded: naruto_v01_ch005_128.jpg
Downloaded: naruto_v01_ch005_129.jpg
Downloaded: naruto_v01_ch005_130.jpg
Downloaded: naruto_v01_ch005_131.jpg
Downloaded: naruto_v01bch005_147.jpg
Downloaded: naruto_v01_ch005_132.jpg
Downloaded: naruto_v01_ch005_133.jpg
Downloaded: naruto_v01_ch005_134.jpg
Downloaded: naruto_v01_ch005_135.jpg
Downloaded: naruto_v01_ch005_136.jpg
Downloaded: naruto_v01_ch005_137.jpg
Downloaded: naruto_v01_ch005_140.jpg
Downloaded: naruto_v01_ch005_138.jpg
Downloaded: naruto_v01_ch005_141.jpg
Downloaded: naruto_v01_ch005_139.jpg
Downloaded: naruto_v01_ch005_143.jpg
Downloaded: naruto_v01_ch005_142.jpg
Downloaded: naruto_v01_ch005_147.jpg
Downloaded: naruto_v01_ch005_146.jpg
Downloaded: naruto_v01_ch005_144_145.jpg
Downloaded: naruto_v01.jpg
Downloaded: naruto_v01_ch001_005.jpg
Downloaded: naruto_v01_ch001_010.jpg
Downloaded: naruto_v01_ch001_007_008.jpg
Downloaded: naruto_v01_ch001_009.jpg
Downloaded: naruto_v01_ch001_012.jpg
Downloaded: naruto_v01_ch001_011.jpg
Dow